In [1]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split

in_train = pd.read_csv("data/training_set_VU_DM_2014.csv")
#in_test = pd.read_csv("data/testing_set_VU_DM_2014.csv")
in_test = pd.read_csv("data/kaggle_test.csv")

in_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [2]:
# Compute median and mean values per query for some numericals
def add_mean_and_median_per_query(df, column):
    grouped = df.groupby("srch_id")[column]
    mean_by_query = grouped.mean().to_dict()
    median_by_query = grouped.median().to_dict()
    
    df[column + "_mean"] = df.srch_id.map(lambda q: mean_by_query[q])
    df[column + "_median"] = df.srch_id.map(lambda q: median_by_query[q])

for df in [in_train, in_test]:
    add_mean_and_median_per_query(df, "price_usd")
    add_mean_and_median_per_query(df, "prop_starrating")
    add_mean_and_median_per_query(df, "prop_review_score")
    
in_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,price_usd_mean,price_usd_median,prop_starrating_mean,prop_starrating_median,prop_review_score_mean,prop_review_score_median
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,0,NaN,0,163.718929,139.05,3.071429,3.0,3.482143,3.5
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,0,NaN,0,163.718929,139.05,3.071429,3.0,3.482143,3.5
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,0,NaN,0,163.718929,139.05,3.071429,3.0,3.482143,3.5
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,5.0,0,NaN,0,163.718929,139.05,3.071429,3.0,3.482143,3.5
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,0,NaN,0,163.718929,139.05,3.071429,3.0,3.482143,3.5


In [3]:
# Downsample a series so that there are as many non-clicked items as clicked and booked
def downsample_series(s):
    s1 = s.loc[s.click_bool == 1]
    s2 = s.loc[s.click_bool == 0]
    
    if (len(s2) == 0):
        return s1
    
    s2 = s2.sample(frac=min(1, len(s1)/len(s)))
    return s1.append(s2).sort_index()

# Split on search ids
srch_ids = in_train["srch_id"].unique()
srch_ids_train, srch_ids_test_val = train_test_split(srch_ids, test_size=0.2, random_state = 42)
srch_ids_test, srch_ids_val = train_test_split(srch_ids_test_val, test_size=0.5, random_state = 42)

_train = in_train.loc[in_train.srch_id.isin(srch_ids_train)]
_val = in_train.loc[in_train.srch_id.isin(srch_ids_val)]
_test = in_train.loc[in_train.srch_id.isin(srch_ids_test)]
_train_undersampled = _train.groupby("srch_id").apply(downsample_series).reset_index(drop=True)

In [5]:
# Save to CSV
_train.to_csv("data/processed/_train.csv", index=False)
_val.to_csv("data/processed/_val.csv", index=False)
_test.to_csv("data/processed/_test.csv", index=False)
_train_undersampled.to_csv("data/processed/_train_undersampled.csv", index=False)
in_test.to_csv("data/processed/kaggle_test.csv")